In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

import warnings

warnings.filterwarnings("ignore")

# Nettoyage des données

In [2]:
#fonction qui importe un fichier csv et renvoie un dataframe

def importer_csv(nom_fichier, chemin):
    chemin_complet = chemin  + nom_fichier 
    dataframe = pd.read_csv(chemin_complet, sep='\t')
    return dataframe

In [3]:
#fonction qui supprime les colonnes vides et des lignes doublons

def supprimer_col_vides_doublons(df):
    df.dropna(axis=1, how='all', inplace=True)
    df.drop_duplicates()
    
    return df

In [51]:
#fonction afficher le % de valeurs nulles pour les colonnes d'un dataframe

def valeurNullesGraph(df):
    
    #pourcentage de valeurs non nulles par colonne
    pourcentage_non_nulles = round(df.notna().mean() * 100,1).sort_values(ascending=False)

    fig = px.bar(x=pourcentage_non_nulles.index,
                 y=pourcentage_non_nulles.values,
                 labels={'x': 'Colonnes', 'y': 'Pourcentage'},
                 title='Pourcentage de valeurs non nulles par colonne',
                 color=pourcentage_non_nulles.values,
                color_continuous_scale='blues')

    fig.update_xaxes(tickangle=45, tickmode='array')
    fig.show()

In [53]:
def suppression_col_correlees(df, seuil_correlation):
    #création d'un dataframe quantitatif
    df_quantitatif = pd.DataFrame()

    for colonne in df.columns:
        if pd.api.types.is_numeric_dtype(df[colonne]):
            df_quantitatif[colonne] = df[colonne]
            
    # Calculer la matrice de corrélation
    matrice_correlation = df_quantitatif.corr()

    # Identifier les paires de colonnes corrélées
    colonnes_correlees = set()
    for i in range(len(matrice_correlation.columns)):
        for j in range(i):
            if abs(matrice_correlation.iloc[i, j]) >= seuil_correlation:
                colonne_i = matrice_correlation.columns[i]
                colonne_j = matrice_correlation.columns[j]
                colonnes_correlees.add((colonne_i, colonne_j))

     # Supprimer la première colonne de chaque paire corrélée
    for colonne1, colonne2 in colonnes_correlees:
        df.drop(columns=colonne1, inplace=True)
    
    
    return df

In [113]:
#fonction qui ne garde que les colonnes pertinentes

def selection_colonnes(df):
    #creation de la liste des variables, on y insérant des variables utiles vis à vis de l'analyse
    variables = ['product_name', 'brands','pnns_groups_1', 'pnns_groups_2', 'main_category_fr', 'nutrition_grade_fr']
    
    #liste de toutes les colonnes en rapport avec les valeurs nutritionnelles
    val_nutri = [col for col in df.columns if col.endswith('_100g')]
    
    #suppression de l'empreinte carbone qui n'affecte pas la valeur nutritionelle, ainsi que le nutriscore UK
    try:
        val_nutri.remove('carbon-footprint_100g')
    except KeyError:
        print('colonne inexistente')
    try:
        val_nutri.remove('nutrition-score-uk_100g')
    except KeyError:
        print('colonne inexistente')
        
    variables = variables + val_nutri
    
    #ne garder que ces variables dans le dataframe
    df = df[variables]
    #liste des colonnes avec +60% de valeurs nulles
    colonnes_a_supprimer = df[val_nutri].columns[round(df[val_nutri].isna().mean()*100,1)> 60]
    
    #suppression de ces colonnes vides ou presque
    df = df.drop(columns=colonnes_a_supprimer)
    
    #suppression d'une colonne si +0.85 de corrélation avec une autre
    df = suppression_col_correlees(df, 0.85)
    
    print('Colonnes pertinentes sélectionnées')
    return df



In [112]:
def valeurs_aberrantes(df):
    
    #remplacer valeurs abérrentes par valeurs nulles
    df['nutrition-score-fr_100g'] = df['nutrition-score-fr_100g'].apply(lambda x: x if (-15 <= x <= 40) else None)  
    df['energy_100g'] = df['nutrition-score-fr_100g'].apply(lambda x: x if (x <= 800) else None)
    
    
    # Sélectionner les colonnes finissant par "100g" sauf "energy_100g" et "nutriscore"
    col_100g = [col for col in df.columns if col.endswith('100g') and col != 'energy_100g' 
                               and col != 'nutrition-score-fr_100g']
    
    for col in col_100g:
        df[col] = df[col].apply(lambda x: x if (0 <= x <= 100) else None)  
    
    
    valeurs_nutriscore = ['a', 'b', 'c', 'd', 'e']
    # Remplacer les valeurs non autorisées par des valeurs nulles
    df['nutrition_grade_fr'] = np.where(df['nutrition_grade_fr'].isin(valeurs_nutriscore), df['nutrition_grade_fr'], None)

    
    print('Valeurs aberrantes remplacées par None')
    
    return df

In [98]:
def suppression_na_ligne(df, liste_colonnes):
    
    df = df.dropna(subset=liste_colonnes)
    
    return df

In [111]:
def remplacer_na_100g_mediane(df, liste_colonnes):
    
    for i in df['main_category_fr'].unique():
        df.loc[df['main_category_fr']== i, liste_colonnes] = df[df['main_category_fr']== i ][liste_colonnes].fillna(df[df['main_category_fr'] == i][liste_colonnes].median()) 
    
    print('Remplacement des valeurs nulles par la médiane de chaque catégorie')
    return df

In [109]:
def remplacer_na_100g_mode(df, liste_colonnes):
     
    for i in df['pnns_groups_2'].unique():
        df.loc[df['pnns_groups_2']== i, liste_colonnes] = df[df['pnns_groups_2']== i ][liste_colonnes].fillna(df[df['pnns_groups_1'] == i][liste_colonnes].mode()) 
    
    print('Remplacement des valeurs nulles par le mode de chaque pnns_group_2')
    return df

In [110]:
def gerer_valeurs_manquantes(df):
    
    print('Suppression des lignes où le nom du produit n est pas renseigné')
    
    #suppression des lignes où le nom du produit n'est pas renseigné
    df = suppression_na_ligne(df, ['product_name'])
    
    col_100g = [col for col in df.columns if col.endswith('_100g')]
    
    #remplacer des Nan par la médiane de la colonne
    df = remplacer_na_100g_mediane(df, col_100g)
    
    col_100g.append('nutrition_grade_fr')
    
    #remplacer des Nan par le mode de la colonne
    df = remplacer_na_100g_mode(df, col_100g)
    
    #suppression des lignes où il reste des valeurs nulles
    df = suppression_na_ligne(df, col_100g)
    
    return df

In [71]:
#défnition variables

chemin = ''
fichier = 'fr.openfoodfacts.org.products.csv'
nom_csv_export = 'sante_publique.csv'

In [114]:
def main():
    df = importer_csv(fichier, chemin)
    df = supprimer_col_vides_doublons(df)
    df = selection_colonnes(df)
    valeurNullesGraph(df)
    df = valeurs_aberrantes(df)
    df = gerer_valeurs_manquantes(df)
    valeurNullesGraph
    
    print(df.shape)
    
    # Exporter le DataFrame dans un fichier CSV
    df.to_csv(chemin + nom_csv_export, index=False)

In [105]:
main()

Colonnes pertinentes sélectionnées


Valeurs aberrantes remplacées par None


Suppression des lignes où le nom du produit n est pas renseigné
Remplacement des valeurs nulles par la médiane de chaque catégorie


Remplacement des valeurs nulles par le mode de chaque catégorie


(173470, 21)


# Analyse

## Univariée

In [106]:
data = pd.read_csv('sante_publique.csv', sep=",")

In [107]:
data.shape

(173470, 21)